In [16]:
import sys

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

sys.path.append(r'C:\Users\19167\AppData\Local\Programs\Python\smartone_quant')

In [4]:
# 获取股票行情数据
"""
"time"                #时间戳
"open"                #开盘价
"high"                #最高价
"low"                 #最低价
"close"               #收盘价
"volume"              #成交量
"amount"              #成交额
"settle"              #今结算
"openInterest"        #持仓量
"""

'\n"time"                #时间戳\n"open"                #开盘价\n"high"                #最高价\n"low"                 #最低价\n"close"               #收盘价\n"volume"              #成交量\n"amount"              #成交额\n"settle"              #今结算\n"openInterest"        #持仓量\n'

In [5]:
from typing import List
import pandas as pd

# 日期工具
from datetime import datetime, timedelta
from xtquant import xtdata

from app.models.qmt_stock_daily import QmtStockDailyOri

In [28]:
"""
get_market_data函数返回数据格式为：
{field1 : value1, field2 : value2, ...}
            field1, field2, ... : 数据字段
            value1, value2, ... : pd.DataFrame 字段对应的数据，各字段维度相同，index为stock_list，columns为time_list
"""


# 公共解析数据的方法，将stock_data解析为股票对象
def parse_stock_data(stock_data: dict, model_cls) -> list:
    """
    解析股票行情数据，支持日/周/月K模型
    :param stock_data: 股票行情数据
    :param model_cls: 股票数据模型类
    :return: 股票对象列表
    """
    stock_list = []
    stock_data_time = stock_data['time']
    stock_codes = stock_data_time.index.tolist()
    stock_time_list = stock_data_time.columns.tolist()
    for stock_code in stock_codes:
        for stock_time in stock_time_list:
            stock_open = stock_data['open'].loc[stock_code, stock_time].round(2)
            stock_high = stock_data['high'].loc[stock_code, stock_time].round(2)
            stock_low = stock_data['low'].loc[stock_code, stock_time].round(2)
            stock_close = stock_data['close'].loc[stock_code, stock_time].round(2)
            stock_volume = stock_data['volume'].loc[stock_code, stock_time]
            stock_amount = stock_data['amount'].loc[stock_code, stock_time]
            stock_obj = model_cls(
                code=stock_code,
                time=stock_time,
                open=stock_open,
                high=stock_high,
                low=stock_low,
                close=stock_close,
                volume=stock_volume,
                amount=stock_amount
            )
            stock_list.append(stock_obj)
    print(f'解析股票数据成功，解析到{len(stock_list)}条数据')

    return stock_list



'\nget_market_data函数返回数据格式为：\n{field1 : value1, field2 : value2, ...}\n            field1, field2, ... : 数据字段\n            value1, value2, ... : pd.DataFrame 字段对应的数据，各字段维度相同，index为stock_list，columns为time_list\n'

In [ ]:
# 获取沪深A股中所有成分股的列表
sector_name: str = '沪深A股'
stock_list_in_sector: List[str] = xtdata.get_stock_list_in_sector(sector_name)
print(f'获取板块成分股列表{len(stock_list_in_sector)}成功：{stock_list_in_sector}')

In [7]:
# 测试下载000001.SZ的日 k 行情数据并获取打印
stock_code: str = '000001.SZ'
print(f'开始下载股票{stock_code}的行情数据')
# 获取一年前的日期,yyyyMMdd格式
one_years_ago = (datetime.now() - timedelta(days=1 * 365)).strftime('%Y%m%d')
print(f'一年前的日期为：{one_years_ago}')
# 获取今天日期
today = datetime.now().strftime('%Y%m%d')
print(f'今天的日期为：{today}')

开始下载股票000001.SZ的行情数据
一年前的日期为：20240613
今天的日期为：20250613


In [7]:
# 下载股票的日K 行情数据
xtdata.download_history_data(
    stock_code=stock_code,
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    incrementally=False)
print(f"下载股票 {stock_code} 的日 K 数据成功")
# 下载股票的周K 行情数据
xtdata.download_history_data(
    stock_code=stock_code,
    period='1w',
    start_time=one_years_ago,
    end_time=today,
    incrementally=False)
print(f"下载股票 {stock_code} 的周 K 数据成功")
# 下载股票的月K 行情数据
xtdata.download_history_data(
    stock_code=stock_code,
    period='1M',
    start_time=one_years_ago,
    end_time=today,
    incrementally=False)
print(f"下载股票 {stock_code} 的月 K 数据成功")

开始下载股票000001.SZ的行情数据
一年前的日期为：20240613
今天的日期为：20250613
下载股票 000001.SZ 的日 K 数据成功
下载股票 000001.SZ 的周 K 数据成功
下载股票 000001.SZ 的月 K 数据成功


In [34]:
# 测试获取下载的行情数据--获取最近10条数据
stock_code: str = '000001.SZ'
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=10,  # 获取最近10条数据
    fill_data=False
)
# 查看返回数据的格式
print(f'获取股票 {stock_code} 的行情数据格式：{type(stock_data)}')
# 解析返回数据
"""
{field1 : value1, field2 : value2, ...}
            field1, field2, ... : 数据字段
            value1, value2, ... : pd.DataFrame 字段对应的数据，各字段维度相同，index为stock_list，columns为time_list
"""
# 获取time字段的数据
stock_data_time: pd.DataFrame = stock_data['time']

stock_data_time.head()  # 打印前5行数据
stock_dailys = parse_stock_data(stock_data=stock_data, model_cls=QmtStockDailyOri)
print(f"获取股票 {stock_code} 的日K行情数据{len(stock_dailys)}成功：\n{stock_dailys[0]}")

获取股票 000001.SZ 的行情数据格式：<class 'dict'>


'\n{field1 : value1, field2 : value2, ...}\n            field1, field2, ... : 数据字段\n            value1, value2, ... : pd.DataFrame 字段对应的数据，各字段维度相同，index为stock_list，columns为time_list\n'

,20250530,20250603,20250604,20250605,20250606,20250609,20250610,20250611,20250612,20250613
000001.SZ,1748534400000,1748880000000,1748966400000,1749052800000,1749139200000,1749398400000,1749484800000,1749571200000,1749657600000,1749744000000


解析股票数据成功，解析到10条数据
获取股票 000001.SZ 的日K行情数据10成功：
time='20250530' open=np.float64(11.47) high=np.float64(11.58) low=np.float64(11.44) close=np.float64(11.56) volume=np.int64(1130849) amount=np.float64(1303433715.9999998) id=None


In [14]:
# 获取下载的行情数据--获取所有数据
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=-1,
    dividend_type='none',  # 不获取分红派息数据
    fill_data=False
)
# 查看返回数据的格式
print(f'获取股票 {stock_code} 的行情数据格式：{type(stock_data)}')
# 通过


获取股票 000001.SZ 的行情数据格式：<class 'dict'>
字段 time 的数据格式：<class 'pandas.core.frame.DataFrame'>
                20240613       20240614       20240617       20240618  \
000001.SZ  1718208000000  1718294400000  1718553600000  1718640000000   

                20240619       20240620       20240621       20240624  \
000001.SZ  1718726400000  1718812800000  1718899200000  1719158400000   

                20240625       20240626  ...       20250530       20250603  \
000001.SZ  1719244800000  1719331200000  ...  1748534400000  1748880000000   

                20250604       20250605       20250606       20250609  \
000001.SZ  1748966400000  1749052800000  1749139200000  1749398400000   

                20250610       20250611       20250612       20250613  
000001.SZ  1749484800000  1749571200000  1749657600000  1749744000000  

[1 rows x 243 columns]
字段 open 的数据格式：<class 'pandas.core.frame.DataFrame'>
           20240613  20240614  20240617  20240618  20240619  20240620  \
000001.SZ     10.89

In [11]:
# 测试未下载的股票行情数据能否获取
stock_code = '603259.SH'  # 测试未下载的股票代码
print(f'开始获取股票 {stock_code} 的行情数据')
xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=10,  # 获取最近10条数据
    dividend_type='none',  # 不获取分红派息数据
    fill_data=False
)

开始获取股票 603259.SH 的行情数据


{'time': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'open': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'high': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'low': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'close': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'volume': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'amount': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'openInterest': Empty DataFrame
 Columns: []
 Index: [603259.SH]}

In [12]:
# 测试能否够获取前复权的股票行情数据
print(f"开始获取股票 {stock_code} 的前复权行情数据")
xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=10,  # 获取最近10条数据
    dividend_type='front',  # 获取前复权数据
    fill_data=False
)

开始获取股票 603259.SH 的前复权行情数据


{'time': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'open': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'high': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'low': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'close': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'volume': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'amount': Empty DataFrame
 Columns: []
 Index: [603259.SH],
 'openInterest': Empty DataFrame
 Columns: []
 Index: [603259.SH]}

In [24]:
# 验证能否获取到未执行下载的日期范围的行情数据，例如获取 两年前，两个月前，两天前，两年前到两个月前的行情数据
stock_code: str = '000001.SZ'
# 获取两年前的日期,yyyyMMdd格式
two_years_ago = (datetime.now() - timedelta(days=2 * 365)).strftime('%Y%m%d')
print(f'两年前的日期为：{two_years_ago}')
# 获取两个月前的日期,yyyyMMdd格式
two_months_ago = (datetime.now() - timedelta(days=2 * 30)).strftime('%Y%m%d')
print(f'两个月前的日期为：{two_months_ago}')
# 获取两天前的日期,yyyyMMdd格式
two_days_ago = (datetime.now() - timedelta(days=2)).strftime('%Y%m%d')
print(f'两天前的日期为：{two_days_ago}')
# 获取两年前到两个月前的所有日K 行情数据
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=two_years_ago,
    end_time=two_months_ago,
    count=-1,  # 获取所有数据
    dividend_type='none',  # 不获取分红派息数据
    fill_data=False
)
# 查看返回数据的格式
print(f'获取股票 {stock_code} 的两年前到两个月前的行情数据格式：{type(stock_data["time"])}')
# 打印获取到的两年前到两个月前的日K 行情数据
print(
    f'获取股票 {stock_code} 的两年前到两个月前的日K行情数据{len(stock_data["time"])}成功：\n{stock_data["time"].head()}')
stock_data["open"].head()

两年前的日期为：20230613
两个月前的日期为：20250413
两天前的日期为：20250610
获取股票 000001.SZ 的两年前到两个月前的行情数据格式：<class 'pandas.core.frame.DataFrame'>
获取股票 000001.SZ 的两年前到两个月前的日K行情数据1成功：
                20230613       20230614       20230615       20230616  \
000001.SZ  1686585600000  1686672000000  1686758400000  1686844800000   

                20230619       20230620       20230621       20230626  \
000001.SZ  1687104000000  1687190400000  1687276800000  1687708800000   

                20230627       20230628  ...       20250328       20250331  \
000001.SZ  1687795200000  1687881600000  ...  1743091200000  1743350400000   

                20250401       20250402       20250403       20250407  \
000001.SZ  1743436800000  1743523200000  1743609600000  1743955200000   

                20250408       20250409       20250410       20250411  
000001.SZ  1744041600000  1744128000000  1744214400000  1744300800000  

[1 rows x 443 columns]


,20230613,20230614,20230615,20230616,20230619,20230620,20230621,20230626,20230627,20230628,...,20250328,20250331,20250401,20250402,20250403,20250407,20250408,20250409,20250410,20250411
000001.SZ,11.76,11.55,11.49,11.61,11.63,11.44,11.36,11.3,11.2,11.28,...,11.39,11.36,11.27,11.25,11.31,11.0,10.68,10.73,10.86,10.87


In [ ]:
# 测试